In [ ]:
import numpy as np
import pandas as pd
import os 
import pickle
import keras 
from keras.datasets import cifar10, mnist
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Conv1D, Conv3D, Input, Activation, Subtract, MaxPool2D, UpSampling2D
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from math import log10, sqrt 
from keras.utils.np_utils import to_categorical  

Using TensorFlow backend.


In [ ]:
#import datasets needed

(train_images_raw,train_label),(test_images_raw,test_label)=mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
#convert labels into one-hot-encoded vectors
def label_vectors():
  train_labels=to_categorical(train_label,num_classes=10)
  test_labels=to_categorical(test_label, num_classes=10)
  return train_labels, test_labels


In [ ]:
def create_image_arrays(train_images,test_images):
  test_images = test_images.astype('float32') / 255.0
  train_images= train_images.astype('float32') / 255.0

  train_images_expanded = np.expand_dims(train_images, axis=3)
  test_images_expanded = np.expand_dims(test_images, axis=3)  
  noisy_train_images=add_gauss(train_images_expanded)
  noisy_test_images=add_gauss(test_images_expanded)

  return train_images_expanded,test_images_expanded,noisy_train_images,noisy_test_images

In [ ]:
#function to add noise
def add_gauss(img):
  noise_added_images=[]
  for i in range(len(img)):
    noise_added_images.append(img[i]+0.7*np.random.normal(0,1.0,(28,28,1)))
  return noise_added_images


In [ ]:
def statistical_filter(img):
  filtered_images=[]
  for i in range(len(img)):
    my_img=img[i]
    X=cv2.GaussianBlur(my_img,(5,5),cv2.BORDER_DEFAULT)
    filtered_images.append(X)
  return filtered_images

In [ ]:
def cnn_denoiser(train_images_noisy,train_images):
  dilation=[1,1,1,1,1,1,1]
  filter_size=[32,32,64,64,128,64,64]
  input_image= Input(shape=(28,28,1))
  x= Conv2D(64,kernel_size=3,activation="relu",dilation_rate=1,strides=(1,1),padding="same")(input_image)
  
  for layers in range(7):
    x=Conv2D(64,kernel_size=3, strides=(1,1),dilation_rate=1,padding="same")(x)
    x= BatchNormalization(axis=-1,epsilon=0.001 )(x)
    x=Activation("relu")(x)

  x=Conv2D(1, kernel_size=3,strides=(1,1),dilation_rate=1,padding="same")(x)
  x=Subtract()([input_image,x])
  model=Model(inputs=input_image, outputs=x)
  #model.compile(optimizer="Adam", loss=["mean_squared_logarithmic_error"])
  model.compile(optimizer="Adam", loss=["mse"])
  
  A=np.array(train_images_noisy)
  B=np.array(train_images)
  model.fit(A,B,epochs=2)
  return model


In [ ]:
def encoder(input_img):
    """
    Encoder function.
    Arguments:
      input_img: Input image, Tensor of shape (28, 28, 1)
    
    Returns: Encoded image, Tensor of shape (7, 7, 8)
    """
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='encoder_conv1')(input_img)
    x = MaxPool2D((2, 2), padding='same', name='encoder_pool1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='encoder_conv2')(x)
    x = MaxPool2D((2, 2), padding='same', name='encoder_pool2')(x)
    encoded = Conv2D(8, (3, 3), activation='relu', padding='same', name='encoder_conv3')(x)
    return encoded


def decoder(encoded):
    """
    Decoder function.
    Arguments:
      encoded: Encoded image, Tensor of shape (7, 7, 8)
    
    Returns: Decoded images, Tensor of shape (28, 28, 1)
    """
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='decoder_conv1')(encoded)
    x = UpSampling2D((2, 2), name='decoder_upsample1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='decoder_conv2')(x)
    x = UpSampling2D((2, 2), name='decoder_upsample2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='decoder_conv3')(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='decoder_conv4')(x)
    return decoded

In [ ]:
def autoencoder_denoiser(x_train_noisy,x_train_expanded,x_test_noisy,x_test_expanded):
  input_img = Input(shape=(28, 28, 1))

  #create autoencoder model
  autoencoder = Model(input_img, decoder(encoder(input_img)))
  #complile model
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  #define callback for earlystopping to stop training if the validation loss does not decrese for 10 epochs 
  earlyStopping = EarlyStopping(monitor='val_loss', patience=2)

  #define callback to save model weights if the validation loss improves
  modelChkpt = ModelCheckpoint(filepath='best_weights.h5', monitor='val_loss', save_best_only=True) 

  #train model
  train_history = autoencoder.fit(x_train_noisy, x_train_expanded,
                epochs=15,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test_expanded),callbacks=[modelChkpt, earlyStopping])
  return autoencoder
  

In [ ]:
##classification model for noisy images
def classification(train_images,train_labels,test_images,test_labels):
  early_stop=[EarlyStopping(patience=2)]
  model=Sequential()
  model.add(Conv2D(32,kernel_size=3,activation='relu', input_shape=(28,28,1)))
  model.add(Conv2D(128, kernel_size=3,activation='relu'))
  model.add(Flatten())
  model.add(Dense(10,activation='softmax'))

  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

  train_images=np.array(train_images)
  test_images=np.array(test_images)
  
  model.fit(train_images,train_labels,validation_data=(test_images,test_labels),epochs=10)
  return model

In [ ]:
####our main space

train_images, test_images, noisy_train_images, noisy_test_images= create_image_arrays(train_images_raw,test_images_raw)

In [ ]:
train_labels, test_labesl=label_vectors()

In [ ]:
autoencoder=autoencoder_denoiser(np.array(noisy_train_images),train_images,np.array(noisy_test_images), test_images)

Epoch 1/15
469/469 [==============================] - 3s 7ms/step - loss: 0.1984 - val_loss: 0.1471
Epoch 2/15
469/469 [==============================] - 3s 6ms/step - loss: 0.1395 - val_loss: 0.1336
Epoch 3/15
469/469 [==============================] - 3s 6ms/step - loss: 0.1322 - val_loss: 0.1287
Epoch 4/15
469/469 [==============================] - 3s 6ms/step - loss: 0.1284 - val_loss: 0.1260
Epoch 5/15
469/469 [==============================] - 3s 7ms/step - loss: 0.1262 - val_loss: 0.1246
Epoch 6/15
469/469 [==============================] - 3s 7ms/step - loss: 0.1248 - val_loss: 0.1231
Epoch 7/15
469/469 [==============================] - 3s 7ms/step - loss: 0.1236 - val_loss: 0.1222
Epoch 8/15
469/469 [==============================] - 3s 6ms/step - loss: 0.1228 - val_loss: 0.1232
Epoch 9/15
469/469 [==============================] - 3s 6ms/step - loss: 0.1222 - val_loss: 0.1215
Epoch 10/15
469/469 [==============================] - 3s 6ms/step - loss: 0.1216 - val_loss: 0.1213

In [ ]:
###journey to the moment of truth begins!
filtered_images_train=statistical_filter(noisy_train_images)
filtered_images_test=statistical_filter(noisy_test_images)

In [ ]:
output_train=autoencoder.predict(np.array(filtered_images_train))
output_test=autoencoder.predict(np.array(filtered_images_test))

In [ ]:
model_noisy=classification(train_images,train_labels,noisy_test_images,test_labesl)

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1120 - accuracy: 0.9667 - val_loss: 2.5498 - val_accuracy: 0.4626
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0399 - accuracy: 0.9877 - val_loss: 4.0873 - val_accuracy: 0.3489
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0244 - accuracy: 0.9922 - val_loss: 4.3364 - val_accuracy: 0.3235
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0155 - accuracy: 0.9948 - val_loss: 5.5161 - val_accuracy: 0.3190
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0112 - accuracy: 0.9962 - val_loss: 8.6699 - val_accuracy: 0.2523
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0079 - accuracy: 0.9973 - val_loss: 7.7881 - val_accuracy: 0.3012
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0060 - accuracy: 0.9980 - val_loss: 7.1441 - val_accuracy:

In [ ]:

model_denoised=classification(train_images,train_labels,output_test,test_labesl)


Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1083 - accuracy: 0.9669 - val_loss: 0.6826 - val_accuracy: 0.7782
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0410 - accuracy: 0.9875 - val_loss: 0.6801 - val_accuracy: 0.7825
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0228 - accuracy: 0.9929 - val_loss: 0.6373 - val_accuracy: 0.7928
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0159 - accuracy: 0.9945 - val_loss: 0.7473 - val_accuracy: 0.7642
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0118 - accuracy: 0.9962 - val_loss: 0.7178 - val_accuracy: 0.7761
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0082 - accuracy: 0.9973 - val_loss: 0.9111 - val_accuracy: 0.7435
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0067 - accuracy: 0.9978 - val_loss: 0.7761 - val_accuracy:

In [ ]:
model_idk=classification(output_train,train_labels,output_test,test_labesl)

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4858 - accuracy: 0.8404 - val_loss: 0.3592 - val_accuracy: 0.8812
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3580 - accuracy: 0.8817 - val_loss: 0.3213 - val_accuracy: 0.8915
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3134 - accuracy: 0.8942 - val_loss: 0.3130 - val_accuracy: 0.8981
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2814 - accuracy: 0.9051 - val_loss: 0.3115 - val_accuracy: 0.8995
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2530 - accuracy: 0.9144 - val_loss: 0.3138 - val_accuracy: 0.8997
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2266 - accuracy: 0.9219 - val_loss: 0.3126 - val_accuracy: 0.9007
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2007 - accuracy: 0.9319 - val_loss: 0.3300 - val_accuracy:

In [ ]:
model_idk_again=classification(noisy_train_images,train_labels,noisy_test_images,test_labesl)

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3875 - accuracy: 0.8744 - val_loss: 0.2566 - val_accuracy: 0.9182
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2039 - accuracy: 0.9345 - val_loss: 0.2628 - val_accuracy: 0.9161
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1108 - accuracy: 0.9639 - val_loss: 0.3002 - val_accuracy: 0.9132
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0428 - accuracy: 0.9862 - val_loss: 0.4521 - val_accuracy: 0.9019
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0272 - accuracy: 0.9908 - val_loss: 0.4729 - val_accuracy: 0.9074
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0236 - accuracy: 0.9917 - val_loss: 0.4951 - val_accuracy: 0.9127
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0186 - accuracy: 0.9933 - val_loss: 0.5448 - val_accuracy:

In [ ]:
model_idks=classification(train_images,train_labels, test_images,test_labesl)

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1115 - accuracy: 0.9663 - val_loss: 0.0500 - val_accuracy: 0.9840
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0419 - accuracy: 0.9870 - val_loss: 0.0367 - val_accuracy: 0.9871
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0247 - accuracy: 0.9918 - val_loss: 0.0403 - val_accuracy: 0.9865
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0162 - accuracy: 0.9949 - val_loss: 0.0499 - val_accuracy: 0.9855
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0097 - accuracy: 0.9967 - val_loss: 0.0435 - val_accuracy: 0.9890
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0081 - accuracy: 0.9970 - val_loss: 0.0646 - val_accuracy: 0.9864
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0068 - accuracy: 0.9976 - val_loss: 0.0593 - val_accuracy:

In [ ]:
import tensorflow as tf
im1 = tf.image.convert_image_dtype(noisy_test_images, tf.float32)
im2 = tf.image.convert_image_dtype(filtered_images_test, tf.float32)
psnr = tf.image.psnr(im1, im2, max_val=1.0)
psnr_train = np.mean(psnr)
print(psnr_train)

3.0908508
